# Il crawler degli appalti da remoto a locale
Il crawler, parte dalle comunicazioni avvenute con successo scaricate da [e01_comunicazioni_json2csv.ipynb](https://github.com/althot/TTD_anac/blob/master/0-etl/e01_comunicazioni_json2csv.ipynb) quindi per ogni url trasferisce l'xml dai vari siti dell'ente al filesystem locale

In [15]:
# Import
import pandas as pd
import numpy as np
import matplotlib
import logging
import urllib
import io
import csv
import time
import os
from lxml import etree

# Constraints
APPALTI_XSD = "http://dati.anticorruzione.it/schema/datasetAppaltiL190.xsd"
INDICE_APPALTI_XSD = "http://dati.anticorruzione.it/schema/datasetIndiceAppaltiL190.xsd"

In [16]:
!head -2 ../data/com_suc.csv

"codiceFiscale";"dataUltimoTentativoAccessoUrl";"esitoUltimoTentativoAccessoUrl";"identificativoPEC";"ragioneSociale";"url"
"84002280547";"2015-04-02T15:21:41.117+0000";"successo";"opec275.20150110123012.28213.03.1.5@pec.actalis.it";"ISTITUTO COMPRENSIVO ""G.G. PONTANO"" CERRETO DI SPOLETO";"http://omnicomprensivocerretodispoleto.it/avcp/trasparenza/11/2014.xml"


In [17]:
# Carica comunicazioni avvenute con successo
com_suc = pd.read_csv('../data/com_suc.csv', sep=';', quoting=csv.QUOTE_ALL)

print ("Numero di url su cui iterare: ",com_suc.shape[0])

Numero di url su cui iterare:  34544


In [18]:
# Ricavare lo schema degli appalti e indice
appalti_xmlschema = etree.XMLSchema(etree.parse(APPALTI_XSD))
indice_xmlschema = etree.XMLSchema(etree.parse(INDICE_APPALTI_XSD))

In [30]:
def get_log(name, force_init=False):
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)

    if force_init==True:
        
        # remove all handler if exist
        for i,v in enumerate(logger.handlers):
            logger.removeHandler(logger.handlers[i])
        
        # delete file log if exist
        try:
            os.remove(name)
        except OSError:
            pass
    # create a file handler only if not exist
    if len(logger.handlers)==0:   
    
        handler = logging.FileHandler(name)
        handler.setLevel(logging.DEBUG)

        # create a logging format
        formatter = logging.Formatter('%(asctime)s;%(name)s;%(levelname)s;%(message)s')
        handler.setFormatter(formatter)

        # add the handlers to the logger
        logger.addHandler(handler)
        
    return logger

In [31]:
# Inizializza logger
logger = get_log(name='e02-appalti-remote2local.log', force_init=False)
logger.handlers


[<FileHandler /Users/tony/Develop/TTD_anac/0-etl/e02-appalti-remote2local.log (DEBUG)>]

In [32]:
#%%timeit

logger.info("=== Start ===")

START=8715
END=10000

# Valorizza a lista di indici su cui iterare
lista_indici=com_suc.iloc[START:END].index.values

In [33]:
for i in lista_indici:
    current_url= com_suc.iloc[i]['url']
    time.sleep( 1 )
    try:
        doc = etree.parse(current_url)
        etree.strip_tags(doc,etree.Comment)
        if appalti_xmlschema.validate(doc):
            # Save doc as counter_so_far.xml
            doc.write('../data/appalti_xml/{}.xml'.format(i))
            logger.info(";%d;appalti;fatto" %(i))
        elif indice_xmlschema.validate(doc):
            logger.info(";%d;indice;saltato" %(i))    
        else:
            logger.info(";%d;schema_errato;saltato" %(i))
    except (urllib.error.URLError, urllib.error.HTTPError, OSError, etree.XMLSyntaxError) as e:
        logger.info(";%d;errore:%s;saltato" %(i, type(e).__name__))